# Avance de proyecto 3: Sistema de recomendación
## Maestría en inteligencia artificial aplicada: Análisis de grandes volúmenes de datos


Equipo:
- Abraham Cabanzo Jimenez A01794355
- Ignacio Antonio Ruiz Guerra A00889972
- Moisés Díaz Malagón A01208580


En esta entrega se condensan los 4 algoritmos presentados en las entregas 1 y 2:
A fin de que pueda compararse su rendimiento y sus aplicaciones de uso.

## Algoritmos

Reescribimos los algoritmos contenidos en las entregas 1 y 2, utilizando programación orientada a objetos y estandarizando entradas y salidas entre ellos para facilitar su uso y para facilitar la evaluación.

In [2]:
# !pip install category_encoders

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import preprocessing_algo1
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process
from collections import Counter
import re


In [12]:
# BD para benchmark de primer algoritmo, basado en técnicas de NLP
df = pd.read_csv("./imdb_top_1000.csv")

# BD para resto de experimentos presentados en las entregas anteriores
ratings = pd.read_csv("ratings.dat", sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv("movies.dat", sep='::', names=['movie_id', 'movie_title', 'genres'], encoding='latin-1')

/var/folders/t3/2fb0wqk504s68c7h3cwcl1fc0000gn/T/ipykernel_93685/27823541.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("ratings.dat", sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'])
/var/folders/t3/2fb0wqk504s68c7h3cwcl1fc0000gn/T/ipykernel_93685/27823541.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("movies.dat", sep='::', names=['movie_id', 'movie_title', 'genres'], encoding='latin-1')


In [13]:

class ContentBasedNLP():
    '''
    Sistema de recomendación basado en contenido.
    Técnicas de NLP, en específico TF-IDF y similaridad coseno.
    '''

    def __init__(self, content):
        nltk.download('stopwords')
        self.stopwords = stopwords.words('english')
        self.content = content
        self.__generate_similarity_mtrx()
        
    def __generate_sentences(self, doc):
        '''
        El preprocesamiento a utilizar consiste en conservar únicamente caracteres alfabéticos, eliminando
        signos de puntuación, caracteres especiales y números.
        Solamente consideremos palabras (tokens) con longitud mayor a un caracter.
        Convertiremos todo a minúsculas.
        Eliminaremos del texto los stopwords del diccionario de stop words de la librería NLTK en inglés.
        '''
        words = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]', ' ', doc)
        words = re.sub(r'\s{2,}', ' ', words.strip())
        words = words.lower().split()
        tokens = [ w for w in words if w not in self.stopwords]
        
        return " ".join(tokens)

    def __generate_similarity_mtrx(self):
        sentences = [self.__generate_sentences(x) for x in self.content['concatenated']]
        tfidfvectorizer = TfidfVectorizer()
        tfidf_matrix = tfidfvectorizer.fit_transform(sentences)
        self.similarity_mtrx = linear_kernel(tfidf_matrix, tfidf_matrix)
        self.movies_index = pd.Series(self.content.index, index=self.content['Series_Title']).drop_duplicates()

    def recommend(self, movie_name):
        idx = self.movies_index['Star Wars']
        # obtenemos el vector de similitudes de esa pelicula con todas las otras
        similarity_scores = list(enumerate(self.similarity_mtrx[idx]))
        # ordenamos las pel[iculas con base en los scores de similaidad
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        # podemos observar las 10 películas más similares, omitimos la primera pues es Star Wars
        similarity_scores = similarity_scores[1:11]
        title_idxs = [i[0] for i in similarity_scores]
        return list(self.content['Series_Title'].iloc[title_idxs].values)

In [14]:
def global_recommender(top_n, movies):
    '''
    Recomendador global de películas.
    Hace recomendaciones generales con base en la popularidad de las películas, las más vistas. En este caso se considerarán como las mas vistas, las más veces calificadas.
    '''
    movieid_to_title = dict(zip(movies['movie_id'], movies['movie_title']))
    top_movies = ratings.value_counts('movie_id').sort_values(ascending=False).head(top_n)
    titles = [movieid_to_title[x] for x in top_movies.tolist()]
    movieids = top_movies.index.tolist()
    return titles, movieids

In [15]:
class CollaborativeFilterBasic():
    """
    Filtro colaborativo básico, basado en la matriz de usuario-item. 
    No toma en cuenta las características de las películas. 
    Funciona bajo la premisa de que a usuarios similares les interesan las mismas cosas.
    Aprende de los intereses de la población. 
    """
    # matriz usuario-item, filas son usuarios, columnas son películas, contenido es la calificación bayesiana
    # es una matriz dispersa

    def __init__(self, movies, ratings):
        self.movies = movies.copy()
        self.ratings = ratings.copy()
        self.__create_user_movie_matrix(self.ratings)
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_title = {v:k for k,v in self.title_to_movieid.items()}
        self.model = None
    
    def find_movieid(self, title):
        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_movieid[closest_match[0]]


    def __create_user_movie_matrix(self, df):
        unique_users = df['user_id'].unique()
        unique_movies = df['movie_id'].unique()

        self.userid_to_idx = dict(zip(unique_users, list(range(len(unique_users)))))
        self.movieid_to_idx = dict(zip(unique_movies, list(range(len(unique_movies)))))

        # mapeos inversos
        self.idx_to_userid = {v:k for k,v in self.userid_to_idx.items()}
        self.idx_to_movieid = {v:k for k,v in self.movieid_to_idx.items()}

        users_idx = [self.userid_to_idx[id] for id in df['user_id']]
        movies_idx = [self.movieid_to_idx[id] for id in df['movie_id']]

        self.user_movie_matrix = csr_matrix((df['rating'], (users_idx, movies_idx)), shape=(len(unique_users), len(unique_movies)))

        # persist to disk
        save_npz('user_movie_matrix.npz', self.user_movie_matrix)

    def train(self, k=10, metric='cosine'):
        self.model = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=metric)
        self.model.fit(self.user_movie_matrix.T)

        
    def recommend(self, movie_title=None, movie_id=None, k=10):
        if movie_id is None:
            movie_id = self.find_movieid(movie_title)
        distances, indices = self.model.kneighbors(self.user_movie_matrix.T[self.movieid_to_idx[movie_id]], n_neighbors=k+1)
        distances = distances.squeeze()
        indices = indices.squeeze()
        movie_list = [self.movieid_to_title[self.idx_to_movieid[idx]] for idx in indices[1:]]
        movies_ids = [self.idx_to_movieid[idx] for idx in indices[1:]]
        
        return movie_list, movies_ids

In [25]:
class ContentBasedBasic():
    '''
    Filtro basado en contenido. No toma en cuenta a los usuarios, sino las características de las películas, pero a diferencia del primer algoritmo, este no utiliza técnicas de NLP. 

    Soporta arranque en frio. 
    '''

    def __init__(self, movies):
        self.movies = movies.copy()
        self.similarity_matrix = None
        self.title_to_idx = dict(zip(self.movies['movie_title'], list(self.movies.index)))
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_idx = dict(zip(self.movies['movie_id'], list(self.movies.index)))
        self.idx_to_title = {v:k for k,v in self.title_to_idx.items()}

    def find_movie_idx(self, title):
        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_idx[closest_match[0]]
    
    @staticmethod
    def __calc_similarity_matrix(movies):
        movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
        genre_counter = Counter(g for genres in movies['genres'] for g in genres)
        # eliminamos las películas que no tienen género
        movies = movies[movies['genres']!='(no genres listed)']
        del genre_counter['(no genres listed)']

        movies['clean_title'] = movies['movie_title'].apply(lambda x: re.sub(r'\s*\([^)]*\)', '', x))
        movies['year'] = movies['movie_title'].apply(ContentBasedBasic.__extract_year)
        movies['decade'] = movies['year'].apply(lambda x: x - x%10)

        # A continuación transformamos a one hot encoding los generos para generar un vector de 1s y 0s para cada película dependiendo si pertenece o no a un género en particular
        genres = list(genre_counter.keys())
        for gender in genres:
            movies[gender] = movies['genres'].transform(lambda x: int(gender in x))

        movie_decades = pd.get_dummies(movies['decade'])
        movie_features = pd.concat([movies[genres], movie_decades], axis=1)

        similarity_matrix = cosine_similarity(movie_features, movie_features)
        return similarity_matrix

    @staticmethod
    def __extract_year(title):
        year = re.search(r'\((\d{4})\)', title)
        return int(year.group(1)) if year else None

    def train(self):
        self.similarity_matrix = self.__calc_similarity_matrix(self.movies)


    def recommend(self, movie_title=None, movie_id=None, top_n=10):
        if movie_id:
            movie_idx = self.movieid_to_idx[movie_id]
        elif movie_title:
            movie_idx = self.find_movie_idx(movie_title)
        sim_scores = list(enumerate(self.similarity_matrix[movie_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:(top_n+1)]
        similar_movies_idx = [i[0] for i in sim_scores]
        similar_movies = [self.idx_to_title[x] for x in similar_movies_idx]
        similar_moviesids = [self.title_to_movieid[x] for x in similar_movies]
        return similar_movies, similar_moviesids

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl

class RatingsDataset(Dataset):
    '''
    Dataset para generar los datos de entrenamiento para el modelo de deep learning.
    Toma como entraada los ratings de las peliculas y transforma las calificaciones existentes en etiquetas de que el usuario vio la pelicula y genera N ejemplos no existentes para etiquetas de que el usuario no vio la pelicula. Esto servirá para entrenar el modelo de deep learning.

    Args:
        ratings: dataframe de pandas con las calificaciones
        negative_examples_qty: cantidad de ejemplos negativos a generar
        all_movieids: lista de IDs de todas las peliculas disponibles, no solo en set de entrenamiento
    Returns:
        tupla con id de usuario, id de pelicula y etiqueta

    '''
    def __init__(self, ratings, negative_examples_qty, all_movieids):
        self.all_movieids = all_movieids
        self.negative_examples_qty = negative_examples_qty
        self.users, self.movies, self.labels = self._get_dataset(ratings, self.all_movieids)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.labels[idx]

    def _get_dataset(self, ratings, all_movieids):
        users, movies, labels = [], [], []

        # hacemos un set de pares, usuario - pelicula, no existen pares duplicados
        user_movie_set = set(zip(ratings['user_id'], ratings['movie_id']))

        for (user_id, movie_id) in user_movie_set:
            users.append(user_id)
            movies.append(movie_id)
            labels.append(1) # consideramos que si calificó la película, la vio
            for idx in range(self.negative_examples_qty):
                # selecciona una pelicula aleatoria
                negative_item = np.random.choice(all_movieids)
                # revisar que el usuario no haya interactuado con esta pelicula antes
                while (user_id, negative_item) in user_movie_set:
                    negative_item = np.random.choice(all_movieids)
                users.append(user_id)
                movies.append(negative_item)
                labels.append(0) # no vio la pelicula

        return users, movies, labels

class DLRecommenderModel(pl.LightningModule):
    '''
    Sistema de recomendcaión que utiliza un modelo de deep learning para implementar un filtro colaborativo que modela una función de similaridad entre vectores (embeddings) de usuarios y películas.

    Args:
        n_users: cantidad de usuarios
        n_movies: cantidad de películas
        ratings: base de datos de calificaiones de peliculas
        all_movieids: lista de IDs de todas las peliculas disponibles, no solo en set de entrenamiento
        negative_examples_qty: cantidad de ejemplos negativos a generar por cada positivo
    '''

    _EMBEDDING_DIMENSIONS = 6

    def __init__(self, n_users, n_movies, ratings, all_movieids, negative_examples_qty):
        super().__init__() # llamamos al constructor de la clase padre
        self.negative_examples_qty = negative_examples_qty
        self.user_embedding = nn.Embedding(num_embeddings=n_users, embedding_dim=self._EMBEDDING_DIMENSIONS) # usamos capa Embedding de pytorch
        self.movie_embedding = nn.Embedding(num_embeddings=n_movies, embedding_dim=self._EMBEDDING_DIMENSIONS)

        # posterior a las capas embeddings se aplican dos capas densas, también llamadas Full connected.
        self.fc1 = nn.Linear(12, 64) # 12 features de entrada = 6 (de user embedding) + 6 (de movie embedding), 64 features de salida o neuronas
        self.fc2 = nn.Linear(64, 32) # 64 features de entrada, 32 features de salida
        self.fc3 = nn.Linear(32, 1) # 32 features de entrada, 1 feature de salida (prediccion)
        self.ratings = ratings.copy()
        self.all_movieids = all_movieids.copy()


    def forward(self, user_id, movie_id):
        # calculamos los embeddings
        user_embedding = self.user_embedding(user_id)
        movie_embedding = self.movie_embedding(movie_id)
        # concatenamos los embeddings de usuario y pelicula
        x = torch.cat([user_embedding, movie_embedding], dim=1)

        # aplicamos las capas densas
        x = nn.ReLU()(self.fc1(x)) # función de activación ReLU para capas intermedias
        x = nn.ReLU()(self.fc2(x))
        out = nn.Sigmoid()(self.fc3(x)) # función de activación sigmoide para obtener un valor entre 0 y 1 como salida final

        return out


    def training_step(self, batch, batch_idx):
        user_ids, movie_ids, labels = batch
        predictions = self(user_ids, movie_ids)
        # utilizaremos la función de pérdida Binary Cross Entropy, pues nuestro problema de clasificación es de tipo binario (dos clases)
        loss = nn.BCELoss()(predictions, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        # utilizamos Adam como optimizador, se puede ajustar el learning rate
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        dataset = RatingsDataset(self.ratings, self.negative_examples_qty, self.all_movieids)
        return DataLoader(dataset, batch_size=512, num_workers=0)

class CollaborativeFilterDL():
    """
    Algoritmo avanzado. Filtro colaborativo basado en deep learning. 
    Utiliza retroalimentación implicita. Es decir, todas las peliculas que un usuario califica se consideran como vistas.
    Mediante el entrenamiento se calculan los embeddings tanto de los usuarios como de las películas así como los pesos de las redes neuronales que modelan la función de similaridad entre los embeddings.  

    Representa un problema de clasificación binaria, donde la etiqueta es 1 si el usuario vio la película y 0 si no la vio. 

    Requiere ser entrenado y actualizado. 
    """

    def __init__(self, ratings, movies):
        self.ratings = ratings.copy()
        self.movies = movies.copy()
        self.__prepare_data()
        self.model = None
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_title = {v:k for k,v in self.title_to_movieid.items()}
        
    def find_movieid(self, title):
        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_movieid[closest_match[0]]


    def __prepare_data(self):
        # utilizaremos los ultimos ejemplos de cada usuario como parte del test set
        self.ratings['latest_rank'] = self.ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
        train_ratings = self.ratings[self.ratings['latest_rank'] != 1]
        test_ratings = self.ratings[self.ratings['latest_rank'] == 1]

        # conservamos unicamente user_id, movie_id, y rating
        self.train_ratings = train_ratings[['user_id', 'movie_id', 'rating']]
        self.test_ratings = test_ratings[['user_id', 'movie_id', 'rating']]

        # Retroalimentación implícita: a todos los ratings se les considera como vistos por el usuario
        self.train_ratings.loc[:, 'rating'] = 1 

    def train(self):
        num_users = self.ratings['user_id'].max() + 1
        num_items = ratings['movie_id'].max() + 1
        self.all_movieids = ratings['movie_id'].unique()

        self.model = DLRecommenderModel(num_users, num_items, self.train_ratings, self.all_movieids, negative_examples_qty=5)

        trainer = pl.Trainer(max_epochs=6, logger=False, accelerator='mps', devices=1)
        trainer.fit(self.model)
        return self.model

    def recommend(self, user_id, model):
        # movieid = self.find_movieid(movie_title)
        not_seen_movies = self.ratings[self.ratings['user_id'] == user_id]['movie_id'].unique()
        not_seen_scores = np.squeeze(model(torch.tensor([user_id]*len(not_seen_movies)), torch.tensor(not_seen_movies)).detach().numpy())

        not_seen_pairs = zip(not_seen_movies, not_seen_scores)
        not_seen_pairs = sorted(not_seen_pairs, key=lambda x: x[1], reverse=True)
        top_recommended = not_seen_pairs[1:11]
        top_recommended = [self.movieid_to_title[x[0]] for x in top_recommended]
        return top_recommended


In [26]:
content_algo1 = preprocessing_algo1(df)
content_nlp = ContentBasedNLP(content_algo1)

collaborative_basic = CollaborativeFilterBasic(movies, ratings)

content_basic = ContentBasedBasic(movies)

collaborative_deeplearning = CollaborativeFilterDL(ratings, movies)

# entrenamos los algoritmos, algunos son entrenamientos no supervisados y otros supervisados
collaborative_basic.train()
content_basic.train()
trained_model = collaborative_deeplearning.train() # almacenamos el modelo entrenado

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moisesdiaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Pruebas de funcionamiento individual de cada algoritmo

In [23]:
content_nlp.recommend('Star Wars') # sistema basado en contenido, usando técnicas de NLP

['Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Lawrence of Arabia',
 'The Bridge on the River Kwai',
 'Star Wars: Episode VII - The Force Awakens',
 'The Ladykillers',
 'When Harry Met Sally...',
 'Kind Hearts and Coronets',
 'Raiders of the Lost Ark',
 'The Fugitive']

In [24]:
collaborative_basic.recommend("Toy Story") # sistema de filtro colaborativo usando KNeighbours

(['Toy Story 2 (1999)',
  'Groundhog Day (1993)',
  'Aladdin (1992)',
  "Bug's Life, A (1998)",
  'Back to the Future (1985)',
  'Babe (1995)',
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Men in Black (1997)',
  'Forrest Gump (1994)',
  'Matrix, The (1999)'],
 [3114, 1265, 588, 2355, 1270, 34, 1196, 1580, 356, 2571])

In [27]:
content_basic.recommend('Aladin') # sistema basado en contenido usando one-hot y similaridad coseno

(['Hercules (1997)',
  'Toy Story (1995)',
  'Lion King, The (1994)',
  'Nightmare Before Christmas, The (1993)',
  'Beauty and the Beast (1991)',
  'All Dogs Go to Heaven 2 (1996)',
  'James and the Giant Peach (1996)',
  'Hunchback of Notre Dame, The (1996)',
  'Aladdin and the King of Thieves (1996)',
  "Cats Don't Dance (1997)"],
 [1566, 1, 364, 551, 595, 631, 661, 783, 1064, 1489])

In [28]:
# user id demo
user_id = 897
collaborative_deeplearning.recommend(user_id, trained_model)

['American Beauty (1999)',
 'Forrest Gump (1994)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Jurassic Park (1993)',
 'Silence of the Lambs, The (1991)',
 'Back to the Future (1985)',
 'Braveheart (1995)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Matrix, The (1999)',
 'Godfather, The (1972)']

## Pruebas de rendimiento

Se realizó la prueba de rendimiento a los algoritmos que se basan en el dataset de MovieLens, pues se requiere de un set de pruebas etiquetado para determinar si el algoritmo recomienda peliculas que en efecto el usuario ha visto. 

Se utilizará el métrico HitRatio@10 que indica la proporción de interacciones en las cuales el algoritmo ha recomendado al menos un elemento correcto (que el usuario ha visto) dentro del top 10 de elementos recomendados.

Tomaremos el set de datos de prueba calculado para el filtro colaborativo basado en deep learning y las interacciones de la base de datos de ratings. 

In [29]:
test_set = collaborative_deeplearning.test_ratings
all_movieids = collaborative_deeplearning.all_movieids
user_interactions = set(zip(test_set['user_id'], test_set['movie_id']))
user_to_seen_movies_dict = ratings.groupby('user_id')['movie_id'].apply(list).to_dict()

In [30]:
from tqdm.notebook import tqdm


def calc_hit_ratio(model_name='collaborative_basic'):
    hits = []
    i = 0
    for (user_id, movie_id) in tqdm(list(user_interactions)[0:100]):
        seen_movies = user_to_seen_movies_dict[user_id]

        if model_name == 'collaborative_deeplearning':
            non_seen_movies = set(all_movieids) - set(seen_movies)
            selected_movies = list(np.random.choice(list(non_seen_movies), 99))
            test_movies = selected_movies + [movie_id]
            predicted_labels = np.squeeze(trained_model(torch.tensor([user_id]*100), torch.tensor(test_movies)).detach().numpy())

            top_10_idx = [test_movies[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()] # i puede ir de 0 a 100, convertimos a indice de pelicula
        elif model_name == 'collaborative_basic':
            for seen_movie in seen_movies:
                # test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
                _, top_10_idx = collaborative_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break
            
        elif model_name == 'content_basic':
            for seen_movie in seen_movies:
                # test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
                _, top_10_idx = content_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break
            # test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
            # _, top_10_idx = content_basic.recommend(collaborative_basic.movieid_to_title[test_movie])
        elif model_name == 'global':
            test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
            _, top_10_idx = global_recommender(top_n=10, movies=movies)


        if movie_id in top_10_idx:
            hits.append(1)
        else:
            hits.append(0)

    return np.mean(hits)*100

In [31]:
print(f"Hit ratio global: {calc_hit_ratio('global'):.2f}%")
print(f"Hit ratio collaborative_basic: {calc_hit_ratio('collaborative_basic'):.2f}%")
print(f"Hit ratio content_basic: {calc_hit_ratio('content_basic'):.2f}%", )
print(f"Hit ratio collaborative_deeplearning: {calc_hit_ratio('collaborative_deeplearning'):.2f}%")


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio global: 0.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio collaborative_basic: 57.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio content_basic: 52.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio collaborative_deeplearning: 42.00%


Conclusiones que se pueden obtener de esta prueba:

1. El recomendador global, basado en popularidad, es el que tiene menor tasa de aciertos, esto se debe a que recomienda las películas más populares, no necesariamente las que le gustan a un usuario en particular.
2. El mejor rendimiento lo obtiene el filtro colaborativo basado en la interacción entre usuarios y películas usando Kneighbors, es rápido de entrenar y al ser un algoritmo no-supervisado no requiere de datos etiquetados. No requiere tampoco de metadatos de películas tal como descripciones o géneros. Por otro lado en tiempo de ejecución es el que tomó más tiempo de cómputo. Otro punto en contra es que este filtro será útil cuando el usuario ya tenga varias interacciones en la plataforma, no en un inicio. 
3. El segundo algoritmo con mejor rendimiento es el basado en contenido. Es de igual forma no supervisado y su matriz de similaridad se calcula rápidamente. Dado que se basa en el contenido de las películas y sus metadatos, no requiere que el usuario tenga interacciones previas, soporta arranque en frio. Por otro lado, otro beneficio es la velocidad de ejecución, al tener la matriz de similaridad pre-calculada la recomendacíon se genera en tiempo constante. 
4. Finalmente el 3er algoritmo con mejor rendimiento es el más complejo de los propuestos, basado en embeddings y redes neuronales. Este algoritmo genera dentro de sus pesos sinápticos una matriz de similaridad que combina usuarios y películas. Su complejidad y el poder de cómputo para su entrenamiento es considerablemente mayor que en el caso de los algoritmos anteriores. La inferencia sin embargo es rápida si se cuenta con hardware especializado como GPUs. En este caso logra un HitRatio@10 de 42%, el cual consideramos que podría incrementar si el algoritmo se entrena con más épocas, pues solo se entrenó en 4 épocas.

